# Procesamiento de datos de Spotify
En este notebook se realizará la consulta y procesamiento de los datos que serán utilizados para el proyecto de "rediseño campaña Spotify Wrapped"

In [1]:
import glob
import pandas as pd
import numpy as np
import time

## Recolección de datos de la primera fuente: Historial de reproducciones extendido
En este caso, los datos del historial de reproducciones extendido son enviados por spotify en una serie de archivos de tipo json, se han seleccionado exclusivamente los archivos con datos del 2023

In [2]:
path = "user_streaming_history"
csv_files = glob.glob(path + "/*.json")
df_list = (pd.read_json(file) for file in csv_files)
big_df   = pd.concat(df_list, ignore_index=True)

In [3]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51189 entries, 0 to 51188
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   ts                                 51189 non-null  object
 1   username                           51189 non-null  object
 2   platform                           51189 non-null  object
 3   ms_played                          51189 non-null  int64 
 4   conn_country                       51189 non-null  object
 5   ip_addr_decrypted                  51189 non-null  object
 6   user_agent_decrypted               51189 non-null  object
 7   master_metadata_track_name         51186 non-null  object
 8   master_metadata_album_artist_name  51186 non-null  object
 9   master_metadata_album_album_name   51186 non-null  object
 10  spotify_track_uri                  51186 non-null  object
 11  episode_name                       3 non-null      object
 12  epis

Al utilizar un DataFrame para representar los datos contenidos dentro de los archivos, se identifica que se cuenta inicialmente con <b>51189 registros</b>, <b>21 campos</b> en los que en algunos casos contamos con valores nulos y que la magnitud de los datos es cercana a los <b>6.8MB</b>

### Transformación de datos
Se identifican los campos ts (timestamp) y ms_played los cuales serán transformados para:
* A partir del timestamp obtener año y mes de reproducción
* Transformar los milisegundos de reproducción a minutos de reproducción

Adicionalmente, se renombran algunas campos para facilidad en su entendimiento

In [4]:
big_df[["ts"]] = big_df[["ts"]].apply(pd.to_datetime)
big_df["year_played"] = big_df["ts"].dt.year
big_df["month_played"] = big_df["ts"].dt.month
big_df["min"] = big_df["ms_played"]/60000
big_df=big_df.rename(columns={"ts": "timestamp", "conn_country": "country", "master_metadata_track_name": "song_name", "master_metadata_album_artist_name": "artist_name", "spotify_track_uri":"track_uri"})

### Limpieza de datos
Se inicia eliminando los campos (columnas) que no se pretende utilizar, como por ejemplo los campos relacionados a episodios o la ip, posterior a esto se eliminan los registros con campos vacios y finalmente se filtran dentro de los registros resultantes aquellos del año 2023 cuyo tiempo de reproducción fue superior a los 30 segundos

In [6]:
column_filtered_df = big_df[["timestamp","year_played","month_played","min","country", "song_name", "artist_name","track_uri"]]
row_filtered_df = column_filtered_df.dropna()
row_filtered_df = row_filtered_df.loc[row_filtered_df["year_played"]==2023]
row_filtered_df = row_filtered_df.loc[row_filtered_df["min"]>(30/60)]

In [7]:
row_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16294 entries, 2448 to 51188
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   timestamp     16294 non-null  datetime64[ns, UTC]
 1   year_played   16294 non-null  int64              
 2   month_played  16294 non-null  int64              
 3   min           16294 non-null  float64            
 4   country       16294 non-null  object             
 5   song_name     16294 non-null  object             
 6   artist_name   16294 non-null  object             
 7   track_uri     16294 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(4)
memory usage: 1.1+ MB


Se identifica que posterior a los procesos de limpieza y transformación, se tienen <b>16294 registros</b>, <b>8 campos</b> (sin valores vacios) y una magnitud cercana a los <b>1.1MB</b>

## Recolección de datos de la segunda fuente: Spotify API
Los datos recolectados desde el API de Spotify serán utilizados para complementar los datos del historial

### Importar la libreria spotipy
La libreria spotipy simplifica la comunicación con el API de Spotify

In [8]:
import sys
!{sys.executable} -m pip install spotipy

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = ""
client_secret = ""
auth_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

### Funciones para obtener datos complementarios
Los datos que se obtendrán serán los siguientes:
* Año de publicación de la canción (del álbum de la canción), para ello es necesario transformar el campo que viene desde el API, puesto que allí puede venir en formato AAAA, AAAA-MM o AAAA-MM-DD
* Popularidad de canción, es una escala que indica si la canción es popular (100) o desconocida (0)
* Género musical, es una categoria en la que puede clasificar la canción, desde el API se obtiene como una lista opcional, para este caso se tomará exclusivamente el primer género de la lista
* Popularidad del artista, es una escala que indica si el artista es popular (100) o desconocido (0)

Los datos se obtienen de dos consultas, la consulta de <b>pistas</b> (canciones) y la consulta de <b>artistas</b>, para optimizar la consulta se utilizarán los métodos que permiten consultas multiples, en donde se realizarán consultas de hasta 30 pistas/artistas en simultaneo (aunque bien la capacidad máxima de los métodos es de 50), la consulta de artistas se hará posterior a la de las pistas para poder obtener el conjunto de artistas asociados a las pistas y no realizar consultas duplicadas

In [10]:
def get_tracks_complementary_info(tracks_uris):
    tracks_df = pd.DataFrame(columns = ["track_uri","artist_uri","year","track_pop"])
    tracks_uris_chunks = [tracks_uris[x:x+30] for x in range(0, len(tracks_uris), 30)]
    for tracks_uris_chunk in tracks_uris_chunks:
        tracks = sp.tracks(tracks_uris_chunk)
        for track in tracks.get("tracks"):
            track_uri = track.get("uri")
            artist_uri = None
            artists = track.get("artists")
            if artists:
                artist_uri = artists[0].get("uri")
            popularity = track.get("popularity")
            album = track.get("album")
            release_date = album.get("release_date")
            year = release_date[0:4] if release_date else None
            tracks_df = pd.concat([tracks_df, pd.Series({"track_uri":track_uri,"artist_uri":artist_uri,"year":year,"track_pop":popularity}).to_frame().T], ignore_index=True)
    return tracks_df

def get_artists_complementary_info(artist_uris):
    artists_df = pd.DataFrame(columns = ["artist_uri","genre","artist_pop"])
    artists_uris_chunks = [artist_uris[x:x+30] for x in range(0, len(artist_uris), 30)]
    for artists_uris_chunk in artists_uris_chunks:
        artists = sp.artists(artists_uris_chunk)
        for artist in artists.get("artists"):
            artist_uri = artist.get("uri")
            genres = artist.get("genres")
            genre = genres[0] if genres else None
            popularity = artist.get("popularity")
            artists_df = pd.concat([artists_df, pd.Series({"artist_uri":artist_uri,"genre":genre,"artist_pop":popularity}).to_frame().T], ignore_index=True)
    return artists_df

def get_complementaries(tracks_uris):
    tracks_df = get_tracks_complementary_info(tracks_uris)
    artist_uris = tracks_df["artist_uri"].unique()
    artists_df = get_artists_complementary_info(artist_uris)
    result = pd.merge(tracks_df, artists_df, on="artist_uri")
    return result

### Listado de canciones
Se requiere obtener el listado unico de canciones, puesto que el dataframe actual contiene el listado de reproducciones y en general una canción puede tener varias reproducciones asociadas

In [11]:
track_uri_list = row_filtered_df["track_uri"].unique()

### Obtención de datos complementarios
A partir del listado unico de URIs se procede a invocar las funciones previamente definidas para generar los registros con los datos complementarios, adicionalmente se toma el tiempo que tarda la función para obtener todos los datos complementarios

In [12]:
start = time.time()
complementaries_df = get_complementaries(track_uri_list)
end = time.time()
print("segundos requeridos para la obtención de datos complementarios: "+str(end-start))

segundos requeridos para la obtención de datos complementarios: 87.77163338661194


In [19]:
complementaries_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5308 entries, 0 to 5307
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   track_uri   5308 non-null   object
 1   artist_uri  5308 non-null   object
 2   year        5308 non-null   object
 3   track_pop   5308 non-null   object
 4   genre       5027 non-null   object
 5   artist_pop  5308 non-null   object
dtypes: object(6)
memory usage: 290.3+ KB


Se identifica que para el caso de los datos complementarios obtenidos, se tienen <b>5308 registros</b>, <b>6 campos</b> (el campo "genero" con valores vacios) y una magnitud cercana a los <b>290.3KB</b>

### Manejo para valores vacios
en este caso, los valores vacios serán completados con el valor "desconocido"

In [20]:
complementaries_df["genre"]=complementaries_df["genre"].fillna(value="desconocido")

## Unificación de datos
Se "combinarán" los datos provenientes de las dos fuentes

In [22]:
base_stream_data_df = pd.merge(row_filtered_df,complementaries_df[["track_uri","year","track_pop","genre","artist_pop"]], on="track_uri")

In [23]:
base_stream_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16294 entries, 0 to 16293
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   timestamp     16294 non-null  datetime64[ns, UTC]
 1   year_played   16294 non-null  int64              
 2   month_played  16294 non-null  int64              
 3   min           16294 non-null  float64            
 4   country       16294 non-null  object             
 5   song_name     16294 non-null  object             
 6   artist_name   16294 non-null  object             
 7   track_uri     16294 non-null  object             
 8   year          16294 non-null  object             
 9   track_pop     16294 non-null  object             
 10  genre         16294 non-null  object             
 11  artist_pop    16294 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(8)
memory usage: 1.6+ MB


Finalmente, se trabajará con <b>16294 registros</b>, <b>12 campos</b> (sin valores vacios) y una magnitud cercana a los <b>1.6MB</b>

## Agregación de registros por canción

In [25]:
df_by_song = pd.pivot_table(
   base_stream_data_df,
   index=["country", "genre"],
   aggfunc={'min': np.sum, 'country': len}
).rename(columns={'country': 'count'})
df_by_song.sort_values(by=['min', 'count'],ascending=False, inplace=True)
df_by_song.reset_index(inplace=True)

In [28]:
df_by_song.head(20)

,country,genre,count,min
0,CO,argentine rock,2098,6780.075767
1,CO,colombian hip hop,1145,2987.979100
2,CO,desconocido,667,1493.632700
3,CO,conscious hip hop,397,1244.160367
4,CO,flamenco urbano,462,1221.466933
5,CO,reggaeton,356,1018.314150
6,CO,boom bap espanol,320,984.570717
7,CO,colombian pop,386,982.592933
8,CO,latin alternative,327,981.623550
9,CO,alternative metal,315,966.167217


In [ ]:
base_stream_data_df.info()